# Notebook 7: Model Selection und Bias-Variance Tradeoff

## Lernziele

* Hyperparameter Tuning: systematisches Ausprobieren verschiedener Hyperparameter.
* Bias/Variance Tradeoff: Analyse der Performance In-Sample und Out-of-Sample in Abhängigkeit der Modellkomplexität.
* k-fold Cross-Validation: Sophistiziertere Art der Validierung um statistische Schwankungen zu reduzieren.
* Bestimmen optimaler Hyperparameter(-kombinationen) mit Hilfe von Cross-Validation.
* Split in Trainings-, Validation-, und Testset.

# Hyperparameter

Hat man einen Machine Learning Algorithmus gewält, muss er üblicherweise noch genauer spezifiziert werden durch Wahl von sogenannten **Hyperparametern**. Beim *kNN-Algorithmus* haben wir solche Hyperparameter schon gesehen: z.B. die Anzahl der zu betrachtenden Nachbarn (das 'k' in kNN) oder die zu verwendende Metrik. Praktisch jeder ML-Algorithmus hat eine Reihe solcher Hyperparameter. 

Einige Beispiele von Hyperparametern verschiedener ML-Algorithmen (nur Auszugsweise, bei weitem nicht abschliessend):
* (Lineare oder Logistische) Regression: Funktionale Form der Schätzgleichung
* Entscheidungsbäume: die Tiefe des Baumes
* Neuronale Netze: die Tiefe und der Aufbau des Netzes, die verwendete Aktivierungsfunktion

Die Hyperparameter werden **nicht** während des Trainingsprozesses (mit der *fit()* Methode von scikit-learn) gelernt, sondern **vor** dem Trainieren mit den Trainingsdaten von Hand **festgelegt**. In scikit-learn setzt man die Hyperparameter, indem man beim Aufruf des verwendenten Machine Learning Modells die Parameter als Argumente übergibt. Werden keine Argumente übergeben, werden sogennante *Default*-Werte übernommen.

## Beispiel zum Hyperparameter Tuning: Klassifikation von simulierten Daten mittels kNN mit unterschiedlicher Anzahl berücksichtigter nächster Nachbarn

Wir verwenden hier einen neuen Datensatz mit simulierten Daten. In diesem Datensatz ist jeder Datenpunkt durch zwei Features 'x1' und 'x2' charakterisiert. Der Label 'y' nimmt entweder die Werte 0 oder 1 an.

In [ ]:
# Im Folgenden werden wir die "üblichen" Pakete benötigen:
import numpy as np # Python Paket zum Umgang mit Vektoren und anderen mathematischen Funktionalitäten
import matplotlib.pyplot as plt # grundlegendes Python Paket um Plots zu erstellen
import pandas as pd # Python Paket zum Umgang mit Datensätzen
import seaborn as sns # Python Paket für "schöne" Plots

In [ ]:
# Hiermit legen wir die Grösse von Plots fest:
plt.rcParams["figure.figsize"] = (10,7)  # in den runden Klammern wird die Breite und Höhe der Plots angegeben

In [ ]:
# Einlesen des Datensatzes als Pandas DataFrame und erste Übersicht
daten = pd.read_csv('SimData.csv', index_col = 0)
daten.head()

In [ ]:
# Graphische Darstellung des Datensatzes
sns.scatterplot(x = 'x1', y = 'x2', data = daten, hue = 'y', legend = False)
# Bemerkung: legend=False "schaltet" die (allenfalls störende) Anzeige der Legende aus.

## Modelle/Modellvarianten definieren
Wir verwenden ein kNN Modell mit 3 nächsten Nachbarn (*model3*) und eines mit 15 (*model15*).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model3 = KNeighborsClassifier(n_neighbors = 3)  # Argument n_neighbors explizit festlegen
model3.get_params()   # model3 ist nun ein kNN-Algorithmus, der 3 nächste Nachbarn berücksichtigt

In [ ]:
model15 = KNeighborsClassifier(n_neighbors = 15)  # Argument n_neighbors explizit festlegen
model15.get_params()   # model15 ist nun eine kNN-Algorithmus, der 15 nächste Nachbarn berücksichtigt

Split des Datensatzes in ein *Trainingsset* (benutzt, um die beiden Modelle zu trainieren) und eine *Validationset* (benutzt, um die Out-of-Sample Performance der Modelle zu ermitteln). Durch Vergleich der Out-of-Sample Performance der beiden Modelle **lernen** wir, welches der beiden Modelle besser geeignet ist.

![TVSplit](TrainValSplit.png)

## Features und Labels definieren und in Trainings- und Validationset splitten

In [ ]:
# Features sind die Spalten 'x1' und 'x2', der Labels ist die Spalte 'y'
X = daten[['x1','x2']].values  # Feature Matrix
y = daten['y'].values   # Vektor der Labels

In [ ]:
# Split in Trainingsset (X_t,y_t) und Validationset (X_v, y_v)
from sklearn.model_selection import train_test_split
X_t, X_v, y_t, y_v = train_test_split(X,y, test_size = 0.25, stratify = y, random_state = 13)

In [ ]:
# Trainieren der beiden Modelle
model3.fit(X_t,y_t)

In [ ]:
model15.fit(X_t,y_t)

In [ ]:
# In-Sample Scores
print('In-Sample:')
print('Score Modell 1:', model3.score(X_t,y_t))
print('Score Modell 2:', model15.score(X_t,y_t))

In [ ]:
# Out-of-Sample Scores
print('Out-of-Sample:')
print('Score Modell 1:', model3.score(X_v,y_v))
print('Score Modell 2:', model15.score(X_v,y_v))

## Analyse der unterschiedlichen Wirkungsweise der beiden Modelle

Um besser zu verstehen, was vor sich geht, eignet sich eine Visualisierung der sogenannten **Decision-Boundary** ("Entscheidungs-Grenze"): sie zeigt, welchen Bereichen im x1-x2-Streudiagramm (d.h. für welche Werte der Features) der Algorithmus welche Klasse zuordnet.

Im Folgenden nutzen wir die **NICHT-prüfungsrelevante** Funktion *plot_dec_bound(clf, X, y)*, um diese Visualisierung zu erstellen. Diese Funktion wird in der nächsten Zelle definiert. Sie müssen diese Funktion **nicht** verstehen. Sie brauchen den Code in dieser Zelle **nicht** durchzulesen. Aber sie **müssen** diese Zelle (wie üblich) mit SHIFT-ENTER **ausführen**. Diese Funktion hat drei Argumente:
* *clf*: das zur Klassifikation zu verwendende Modell
* *X*: die Features der Daten des Trainingssets
* *y*: die Labels der Daten des Trainingssets

In [ ]:
# Hier wird die Funktion plot_dec_bound(clf, X, y) definiert
# Diese Zelle ist NICHT prüfungsrelevant!
# Sie müssen den Code in dieser Zelle NICHT durchgehen.
# Hilfsfunktion zur Visualisierung der Decision Boundary
# import matplotlib.pyplot as plt
# import numpy as np
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import LabelEncoder
def plot_dec_bound(clf, X, y):
    LE = LabelEncoder()
    LE.fit(y)
    cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF']) #, '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#0000FF']) # , '#0000FF'])

    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = LE.transform(clf.predict(np.c_[xx.ravel(), yy.ravel()]))
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading = 'auto')

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=LE.transform(y), cmap=cmap_bold, s = 10)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("Decision Boundary")
    plt.show()

Wir schauen uns nun mit Hilfe der soeben definierten Funktion die Streudiagramme und **Decision-Boundaries** der beiden Modelle an.

In [ ]:
# Trainingsset und resultierende Decision-Boundary des kNN-Modells mit 3 nächsten Nachbarn
plot_dec_bound(model3, X_t, y_t)

In [ ]:
# Trainingsset und resultierende Decision-Boundary des kNN-Modells mit 15 nächsten Nachbarn
plot_dec_bound(model15, X_t, y_t)

**Interpretation**

* Die Decision-Boundary des Modells mit 3 nächsten Nachbarn ist deutlich unruhiger oder zackiger als beim Modell mit 15 nächsten Nachbarn. Einbezug von mehr nächsten Nachbarn *glättet* also die Decision-Boundary.
* Die Decision-Boundary des Modells mit 3 nächsten Nachbarn identifiziert etliche "blaue" Ausreisser innerhalb des "roten" Gebiets. Das führt zu einer besseren In-Sample Performance. Allerdings werden neue Datenpunkte dadurch wahrscheinlich öfters falsch klassifiziert, was zu einer kleineren Out-of-Sample Performance führt.
* Insgesamt sieht die Decision-Boundary des Modells mit 15 nächsten Nachbarn "vernünftiger" aus. Wenn wir "von Hand" eine Decision-Boundary in das Streudiagramm der Trainingsdaten hätten einzeichnen müssen, dann hätten wir das wahrscheinlich eher so wie beim Modell mit den 15 Nachbarn gezeichnet. Diese Grenze scheint gut die Gebiete in denen *üblicherweise* (bis auf einzelne *Ausreisser*) die blauen und roten Punkte liegen zu trennen.

## Hyperparameter Tuning: Systematisches Ausprobieren vieler verschiedener Modelle und/oder Parameterwerte

Oben haben wir für den Hyperparameter zwei mögliche Werte eingesetzt und gesehen, dass das eine Modell bessere Resultate liefert als das andere. Ziel des Hyperparameter Tuning ist es, den Spielraum der möglichen Hyperparameter-Werte möglichst systematisch und vollständig auszuloten, um denjenigen zu finden, der zu den besten Resultaten führt.

In unserem Fall bedeutet dies, dass wir systematisch Analysieren wollen, welche Anzahl nächster Nachbarn die beste Out-of-Sample Genauigkeit liefert. Wir gehen daher eine ganze **Liste** von potentiell sinnvollen Parameterwerten durch.

In [ ]:
# Definition der Liste der auszuprobierenden Werte für den Parameter n_neighbors.
klist = [1,3,5,7,9,11,13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 41, 45, 49, 57, 65, 73, 89, 105]

Wir benutzen nun eine Python *for*-Schleife um diese Liste der Parameter durchzugehen und für jeden Wert
- das kNN-Modell mit der entsprechenden Anzahl nächster Nachbarn definieren
- das Modell auf dem Trainingsset trainieren
- die In-Sample (auf dem Trainingsset) und Out-of-Sample (auf den Validationset) Performance messen

Die Perormance Kennzahlen der verschiedenen Modelle sammeln wir in zwei Listen *insamp* und *outsamp*, die wir beim Durchlaufen der Schleife nach und nach aufbauen.

In [ ]:
# Vorbereitung der Listen, in denen die Resultate der In- und Out-of-Sample Genauigkeit gesammelt werden sollen
insamp = []  # Liste für die In-Sample Genauigkeit
outsamp = [] # Liste für die Out-of-Sample Genauigkeit

In [ ]:
# In einer for-Schleife die Werte der Hyperparameter durchgehen und die Resultate (In- und Out-of-Sample scores)
# in den Listen insamp (für Trainingsset) und outsamp (für Testset) sammeln
for k in klist:
    model = KNeighborsClassifier(n_neighbors = k)  # Modell definiern
    model.fit(X_t,y_t)                             # Modell auf dem Trainingsset trainieren 
    insamp.append(model.score(X_t,y_t))         # resultierende In-Sample Performance an die Liste insamp anhängen
    outsamp.append(model.score(X_v,y_v))        # resultierende Out-of-Sample Performance an die Liste outsamp anhängen

## Auswertung der Ergebnisse

Wir schaen uns zunächst die Performancekennzahlen (Accuracy) der verschiedenen Modellvarianten an.

In [ ]:
# In-Sample Accuracies
insamp

In [ ]:
# Out-of-Sample Accuracies
outsamp

Anstatt die Zahlen direkt zu betrachten, können wir viel verstehen, was vor sich geht, wenn wir die In- und Out-of-Sample Accuracy in einem Linienplot (*sns.lineplot(x,y)*) als Funktion der Anzahl nächster Nachbarn darstellen. Auf der x-Achse sind die verwendeten Parameterwerte (k=1 bis k=105) abgetragen. Auf der y-Achse ist die resultierende In-Sample (blaue Linie) und Out-of-Sample (rote Linie) Vorhersagegenauigkeit abgetragen.

In [ ]:
sns.lineplot(x = klist, y = insamp, color ='blue')   # Linie der In-Sample scores
sns.lineplot(x = klist, y = outsamp, color = 'red')  # Linie der Out-of-Sample scores

Mit *matplotlib* Funktionen können wir dieses Diagramm noch "verschönern" indem wir die Achsen beschriften und einen Titel hinzufügen.

In [ ]:
# Mit matplotlib.pyplot Funktionen können wir das Diagramm noch beschriften
# import matplotlib.pyplot as plt
plt.title('In-Sample (blau) und Out-of-Sample (rot) Vorhersagegenauigkeit') # Titel des Diagramms
plt.xlabel('Anzahl Nachbarn') # Titel der x-Achse
plt.ylabel('Genauigkeit') # Titel der y-Achse
sns.lineplot(x = klist, y = insamp, color ='blue')
sns.lineplot(x = klist, y = outsamp, color = 'red')

**Beobachtungen**:

* Die Out-of-Sample Genauigkeit ist (praktisch) immer kleiner als die In-Sample Genauigkeit.
* Die **In-Sample Genauigkeit** zeigt einen (ziemlich) **monotonen** Verlauf: Je komplexer das Modell (je kleiner das k), desto besser die In-Sample Genauigkeit. Im Extremfall k=1 erreichen wir 100% In-Sample Genauigkeit.
* Die **Out-of-Sample Genauigkeit** steigt, erreicht ein **Maximum** und fällt dann wieder. "Extreme" Modelle (mit sehr kleinem k oder sehr grossem k) führen **beide** zu einer schlechten Out-of-Sample Genauigkeit. Das beste Modell mit der grössten Out-of-Sample Genauigkeit liegt irgendwo **in der Mitte**. In diesem Beispiel bei k ingendwo zwischen etwa 15 und 20.

## Detaillierte Analyse der In-Sample und Out-of-Sample Performance als Funktion des Hyperparameter k (Anzahl nächster Nachbarn)

Wir wollen den Verlauf der Performance-Kurven im obigen Bild besser verstehen. Dazu sehen wir uns **drei** Fälle genauer an:
* *model_Over*: Kleinster verwendeter Hyperparameter *n_neighbors=1*
* *model_Under*: Grösster verwendeter Hyperparameter *n_neighbors=105*
* *model_Opt*: Modell mit der grössten Out-of-Sample Genauigkeit.

#### (Halbautomatisches) Finden des Modells mit der besten Out-of-Sample Genauigkeit
Für das *model_Opt* müssen wir zunächst das k mit der grössten Out-of-Sample Genauigkeit finden. Dieses k könnte man natürlich "von Hand" aus der obigen Grafik ablesen, aber man kann es auch mit Hife entsprechender Python/numpy-Funktionen "automatisch" finden.

In [ ]:
# Zunächst die LISTE der Out-of-Sample Genauigkeiten outsamp in einen NUMPY nd.array konvertieren
outarray = np.array(outsamp)
outarray

In [ ]:
# Mit der Methode max() finden wir in einem ndarray den grössten Wert
print('Grösster Wert im Array:', outarray.max())

In [ ]:
# Mit der Methode argmax() finden wir die INDEXPOSITION des grössten Wertes im Array
print('Indexposition des grössten Wertes im Array:', outarray.argmax())

Jetzt müssen wir nur noch herausfinden, welchen Wert für k wir an dieser Indexposition ausprobiert haben:

In [ ]:
# Die Liste der ausprobierten k-Werte:
klist

In [ ]:
# Der k Wert des Modells an der Indexposition 6
klist[outarray.argmax()]

In [ ]:
# Zusammenfassung
print('Grössste erreichte Out-of-Sample Genauigkeit:', outarray.max())
print('Modell mit der besten Out-of-Sample Genauigkeit: k =', klist[outarray.argmax()])

### Analyse der drei Modelle

In [ ]:
# Modelle definieren
model_Over = KNeighborsClassifier(n_neighbors = 1) # kleinstes k
model_Under = KNeighborsClassifier(n_neighbors = 105) # grösstes k
model_Opt = KNeighborsClassifier(n_neighbors = 13) # "optimales" k

In [ ]:
# Modelle auf Trainingsset trainieren
model_Over.fit(X_t, y_t)
model_Under.fit(X_t, y_t)
model_Opt.fit(X_t, y_t)

In [ ]:
# Modell mit k = 1
plot_dec_bound(model_Over, X_t, y_t)

In [ ]:
# Modell mit k = 105
plot_dec_bound(model_Under, X_t, y_t)

In [ ]:
# Modell mit k = 13
plot_dec_bound(model_Opt, X_t, y_t)

**Beobachtungen**:
* Je kleiner das k, desto komplexere Strukturen kann das Modell nachbilden: ein **kleines k** entspricht also einem **komplexen Modell**, ein **grosses k** einem **einfachen Modell**. Modelle mit kleinem k zeigen eine wild gezackte, aufgesplitterte Decision-Boundary. Modelle mit (sehr) grossem k zeigen eine ganz glatte Decision-Boundary.
* **Problem** bei sehr einfachen Modellen (sehr grosses k, hier k = 105): sie sind **zu unflexibel** und können die in den (Trainings-)Daten enthaltenen Strukturen **nicht gut** nachbilden. Man spricht hier von **Underfitting** bzw. sagt zu einfache Modelle haben einen **grossen Bias** (d.h. "sie liegen tendentiell immer falsch").
* **Problem** bei sehr komplexen Modellen (sehr kleines k, hier k = 1): sie sind **zu flexibel** und bilden **alle** Strukturen in den Trainingsdaten (exakt) nach, **auch** das in den Traniningsdaten enthaltene **Rauschen (Noise)**. Jeder *Ausreisser* wird als *systematisch* empfunden. Das *Noise*/die *Ausreisser* sind aber von Datensatz zu Datensatz völlig unterschiedlich, und insbesondere im Validation-Set auch völlig anders als im Trainings-Set. Deshalb die schlechte Out-of-Sample Performance dieser Modelle. Man spricht hier von **Overfitting** bzw. sagt, zu komplexe Modelle haben eine **hohe Varianz** (d.h. "sie zeigen von Sample zu Sample völlig unterschiedliches Verhalten").
* Das **beste Modell** ist das mit der **grössten Out-of-Sample Genauigkeit** und liegt (wie so oft) irgendwo in der Mitte (hier bei k = 13). Es hat meist einen *mittleren* Komplexitätsgrad: genügend komplex, um Strukturen die von Datensatz zu Datensatz reproduzierbar sind, nachzubilden; aber genügend einfach, um nicht auf das nicht-reproduzierbare Noise 'hineinzufallen'. Man spricht hier von einem optimalen **Tradeoff** zwischen Overfitting und Underfitting, bzw. einem optimalen Tradeoff zwischen möglichst kleinem **Bias** und möglichst kleiner **Variance**. Dies nennt man den **Bias-Variance-Tradeoff**.

## Grosses Achtung:

### In-Sample Overfitting (also die Wahl von Modellen mit hoher Varianz) ist einer der am häufigsten gemachten Fehler im Machine Learning!

## Diskussion

Wir beurteilen die im ersten Teil untersuchten beiden Modelle mit 3 und mit 15 nächsten Nachbarn anhand folgender Kriterien:

Das *model3* bzw. *model15*...
- ist komplex/einfach/optimal
- zeigt overfitting/underfitting/optimalen Tradeoff zwischen over- und underfitting
- ist zu flexibel/zu unflexibel/gerade richtig
- hat eine grosse Varianz/kleine Varianz
- hat einen grossen Bias/kleinen Bias
- bildet nicht-reproduzierbares Noise nach/nicht nach
- kann die reproduzierbaren Strukturen in den Daten abbilden/nicht abbilden

## "Stabilere" Ergebnisse durch Cross-Validation

Meist unterliegen die ermittelten Performance Kennzahlen (z.B. die Vorhersagegenauigkeit) im Validationset spürbaren statistischen Schwankungen (siehe z.B. das recht wilde 'hin-und-her-zappeln' der Out-of-Sample Vorhersagegenauigkeit als Funktion des Hyperparameter k, die rote Linie, im Bild oben). Gleichzeitig sind die Unterschiede in der Out-of-Sample Performance bei unterschiedlichen Hyperparametern oft recht klein, so dass man oft kaum unterscheiden kann, ob eine scheinbar bessere Performance rein auf statistischen Schwankungen beruht oder tatsächlich 'nachhaltig' ist.

Die statistischen Schwankungen liessen sich am besten reduzieren, wenn man mehr Daten hätte. Leider ist es aber oft nicht möglich, an (substantiell) mehr Daten zu kommen. Eine Lösung, die mit den verfügbaren Daten auskommt, ist die sogenannte **Cross-Validation**.

### Beispiel: 5-fache Cross-Validation ("5-fold Cross-Validation")
Das Sample wird in 5 gleich grosse Teile aufgeteilt. Es werden nun 5 Durchläufe (*Runs*) durchgeführt; d.h. es wird fünf mal auf dem Trainingsteil (vier fünftel der Daten) trainiert und dann die Out-of-Sample Performance auf dem Validationset (das übrige fünftel der Daten) gemessen. Bei jedem Run wird ein anderes Füntel des Datensatzes als Validationset genommen. Im Bild unten sind die 5 Teile des Datensatzes und die 5 Runs gezeigt. Der gelbe Block repräsentiert dabei für jeden Run jeweils das Validationset, die grünen Teile das Trainingsset.

Als (Zwischen-)ergebnis erhält man für jeden Run die In-Sample (im Trainingsset) und Out-of-Sample (im Validationset) Performance. Schlussendlich werden jeweils die fünf In-Sample und die fünf Out-of-Sample Performance Kennzahlen **gemittelt**, um eine *präzisere* Schätzung der In- und Out-of-Sample Performance zu erhalten.

![CVSplit](CrossVal5.png)

**Umsetzung in scikit-learn**:

Die scikit-learn Funktion für die Cross-Validierung ist *cross_validate()*. Diese Funktion führt **völlig automatisch** das oben beschriebene Prozedere durch.

In [ ]:
# Die Funktion aus scikit-learn importieren
from sklearn.model_selection import cross_validate

Wir spielen die Cross-Validation einmal im Detail durch für ein kNN-Modell mit $k = 3$ Nachbarn.

In [ ]:
# Das Modell definieren
model = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
# So benutzt man die Funktion zur Cross-Validation
scores = cross_validate(model, X, y, cv = 5, return_train_score = True)
scores

Für uns wichtige Parameter der Funktion *cross_validate()*:

* erster Parameter: das zu evaluierende Modell, hier *model*
* zweite und dritte Parameter: Feature-Matrix und Labels des (gesamten) Samples, hier X und y
* *cv=*: Die Anzahl der Splits, hier *cv = 5*, d.h. Aufteilen des gesamten Samples in 5 Teile (Splits)
* *return_train_score=*: Falls "True" gibt die Funktion sowohl die Out-of-Sample (auf dem Validationset berechnete) **als auch** die In-Sample (auf dem Trainingsset berechnete) Performance zurück

Als Ergebnis liefert die Funktion *cross_validate()* ein **Dictionary** mit den für die Berechnung benötigten Rechenzeiten (für uns irrelevant) und den Trainings- und Test-Scores. Alle Einträge in diesem Dictionary sind numpy ndarrays, die für jeden der fünf Durchläufe die entprechenden Grössen (Rechenzeiten/Scores) festhalten.

**Bemerkung**: Standardmässig liefert die Funktion *cross_validate()* in den Teilen *test_score* und *train_score* dasjenige Performance Mass, welches durch die *score()* Methode des Modells berechnet wird. Bei Klassifikationsmodellen (wie z.B. hier das kNN-Modell) ist das üblicherweise die Vorhersageganauigkeit (accuracy).

**Beachten Sie**: Innerhalb der Funktion *cross_validate()* werden mehrere Schritte abgearbeitet:
- die als Parameter übergebenen Features und Labels werden in 5 Teile gesplittet,
- es werden die oben genannten 5 Runs durchgeführt
- für jeden Run wird das als Parameter übergebene Modell auf dem Trainigsteil trainiert,
- für jeden Run wird auf dem Trainings- und Validationteil die Performance gemessen.

### Auswertung:

In [ ]:
# Zunächst aus dem Dictionary 'scores' die 5 Out-of-Sample Scores der 5 Runs als numpy array auslesen
scores['test_score']

Der **Mittelwert** der scores der einzelnen fünf Durchläufe ist der *cross-validated* score:

In [ ]:
# Den Mittelwert der 5 Out-of-Sample scores bestimmen wir mit der numpy Methode mean()
scores['test_score'].mean()

**Zusammenfassung**: die *cross-validated* In- und Out-of-Sample Performance

In [ ]:
# Cross-validated In- und Out-of-Sample Scores
print('In Sample Score:', scores['train_score'].mean())
print('Out-of-Sample Score:', scores['test_score'].mean())

### Exkurs: Cross-Validation bei *geordneten* Datensätzen.

Mit dem Argument 'cv = 5' in der Funktion *cross_validate()* wird der Datensatz **genau der Reihe nach** in die 5-Teile gesplittet (das erste 5tel des Datensatzes kommt in den ersten Teil, das zweite in den zweiten etc.). Wenn die Daten im Datensatz **ungeordnet**, gut durchmischt vorliegen, ist dieses Vorgehen korrekt. Sind die Daten im Datensatz jedoch **geordnet** (z.B. der Grösse einer Variablen nach, oder nach Klassen), so liefert ein Split der Reihe nach **falsche** Ergebnisse.

Die **Lösung bei geordneten Datensätzen** ist, diese **vor dem Split zu mischen**. Dies geschieht durch die Angabe des Parameters 'cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 5)'. Beispiel:

*scores = cross_validate(model, X, y, cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 5), return_train_score = True)*

Die Anzahl der Splits und der 'random_state' können dabei natürlich beliebig festgelegt werden.

## Vollständige Analyse mittels Cross-Validation

Wir überprüfen heir nochmals die Qualität von kNN Modellen mit Hyperparameter k zwischen 1 und 105, wobei wir aber *Cross-Validation* verwenden, um die Modell-Performance zu ermitteln.

In [ ]:
# Hier gehen wir wie oben die Liste der auszuprobierenden Hyperparameter k in einer Schleife durch
# und sammeln die durch Cross-Validation ermittelten Performance Kennzahlen in zwei Listen insamp_cv und outsamp_cv
insamp_cv = []
outsamp_cv = []
for k in klist:
    model = KNeighborsClassifier(n_neighbors = k)
    scores = cross_validate(model, X, y, cv = 5, return_train_score = True) # Wir verwenden hier 5-fache CV
    insamp_cv.append(scores['train_score'].mean())
    outsamp_cv.append(scores['test_score'].mean())

In [ ]:
# Visualisierung der Ergebnisse
plt.title('In-Sample (blau) und Out-of-Sample (rot) Cross-Validated Genauigkeit (dicke Linien)')
plt.xlabel('n_neighbors = k')
plt.ylabel('Genauigkeit')
# Die Cross-validated Scores
sns.lineplot(x = klist, y = insamp_cv, color ='blue', linewidth = 2)
sns.lineplot(x = klist, y = outsamp_cv, color = 'red', linewidth = 2)
# Zum Vergleich, als dünne Linien noch die Scores von oben, ohne Cross-Validation
sns.lineplot(x = klist, y = insamp, color ='blue', linewidth = 0.5)
sns.lineplot(x = klist, y = outsamp, color = 'red', linewidth = 0.5)

**Beobachtung**:

- Der Verlauf der *cross-validated* In-Sample und ganz besonders der Out-of-Sample Scores erscheint nun deutlich *glatter/gleichmässiger* als die mit einem einzigen Validationset ermittelten Werte. Durch die Cross-Validation wurden die statistischen Fluktuationen etwas reduziert, was auch das Ziel war. Die bereits oben erkannte Struktur bleibt dabei erhalten und wird deutlicher sichtbar.
- Wir sehe auch, dass es wohl kein eindeutiges, otpimales k gibt. Gute Werte für k sind wohl alle (ungeraden) Werte zwischen etwa 19 und 29.

In [ ]:
# Das k des Modells mit der besten Out-of-Sample Cross-Validated Performance
klist[np.array(outsamp_cv).argmax()]

# Abschluss: Train-, Validation- und Testset

Oben haben wir die auf dem Validationset ermittete 'Out-of-Sample' Performance zum **Lernen** der Hyperparameter benutzt. Da damit die im Validationset vorhandenen Daten bereits im Lernprozess verwendet wurden, ist die auf dem Validationset gemessene Out-of-Sample Performance **kein guter Indikator** dafür, wie der am Schluss gewählte Algorithmus auf **gänzlich neuen** Daten performen wird. Um eine gute Schätzung für die Performance auf neuen Daten zu erhalten, müssten Daten vorhanden sein, die **in keinster Weise** im Lernprozess verwendet wurden.

Das vollständige Vorgehen (Umgang mit den Vorhandenen Daten) ist daher Folgendes:

* die vorhandenen Daten zunächst in **zwei** Teile aufteilen: einen Teil, der im Lernprozess verwendet wird, und einen Teil, der - ohne jemals angeschaut zu werden - als **Testset** bis zum Schluss 'zur Seite gelegt wird'.
* der Datensatz für den Lernprozess wird wie oben genutzt, um das Modell zu fitten (auf den Trainingsdaten) und ggf. Hyperparameter zu lernen (mittels Validationset oder Cross-Validation).
* Ist der Lernprozess **vollständig abgeschlossen** wird die Performance des Modells auf den bis hierhin noch gänzlich unbenutzten Daten des Testsets gemessen. Die auf dem Testset ermittelte Performance ist damit eine guter Schätzwert für die Performance des Modells auf neuen Daten.

![tvt](TrainValTestSplit.png)